# Create TLUSTY sed grids for speedysedfit by useing the spectrum downloaded from Spectral Energy Distributions (SEDs) of [OTLUSTY2002](http://tlusty.oca.eu/Tlusty2002/tlusty-frames-OS02.html) and [BSTAR2006](http://tlusty.oca.eu/Tlusty2002/tlusty-frames-BS06.html)

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import joblib
import numpy as np
from glob import glob
from speedysedfit.tlustysed import rsed
from speedysedfit.broadsed import vgconv
from tqdm import tqdm
import os

In [2]:
clight = 299792.458

def get_primaryhdu(LOGZ='G', REF='TLUSTY2002', Z = 0.):
    n = np.zeros((1, 2), dtype=np.int32)
    primary_hdu = fits.PrimaryHDU(n)
    header = primary_hdu.header
    header['LOGZ'] = (LOGZ, 'Metallicity wrt Solar')
    header['REF'] =(REF, 'Table reference')
    header['WAVUNIT'] = ('angstrom', 'wavelength units')
    header['FLXUNIT'] = ('erg/s/cm2/A', 'flux units')
    header['Z'] = Z
    return primary_hdu

def get_tablehdu(fname, R =300, wave_start= 9.09e+01, wave_end = 1.60e+06):
    '''
    R: [float] is resolution
    '''
    extname = os.path.basename(fname)
    _extname = extname.replace('B', '')
    TEFF = float(_extname[1:6])
    LOGG = float(_extname[7:10])/100
    Vt = float(_extname[11:13])
    vfhmw = clight/R
    wcor, fcor = rsed(fname)
    w1, fbin = vgconv(wcor, fcor,vfhmw, ppr=1)
    ind =( w1 >= wave_start) & (w1 <=wave_end )
    _w1 = np.array(w1[ind], dtype=np.float32)
    _fbin = np.array(fbin[ind], dtype = np.float32)
    cw = fits.Column(name='wavelength', array=_w1, format='E')
    cf = fits.Column(name='flux', array=_fbin, format='E')
    table_hdu = fits.BinTableHDU.from_columns([cw,cf])
    header = table_hdu.header
    header['EXTNAME'] = (extname, 'name of the extension')                          
    header['TEFF']    = (TEFF, 'Effective temperature (K)')                
    header['LOGG']    = (LOGG, 'Log(g)')
    header['Vt']    = (Vt, 'microturbulence velocity (km/s)')
    table_hdu.header = header
    return table_hdu

# Solar Metallicity grids of OTLUSTY2002

In [19]:
filelist = glob('/share/lijiao/TLUSTY_grid/wangluqian/Tlusty/G*.flux')
dirout = '/share/lijiao/speedyfit/modelgrids/'
primary_hdu = get_primaryhdu(LOGZ='G', REF='OTLUSTY2002', Z = 0.)
table_hdus = joblib.Parallel(n_jobs=10, )(joblib.delayed(get_tablehdu)(fname, R =300, wave_start= 9.09e+01, wave_end = 1.60e+06) for fname in tqdm(filelist))

hdul = fits.HDUList([primary_hdu] + table_hdus)
hdul.writeto(dirout+'TlustyO_G_v10.fits', overwrite=True)

100%|██████████| 69/69 [00:04<00:00, 15.24it/s]


# Solar Metallicity grids of BTLUSTY2006

In [4]:
filelist = glob('/share/lijiao/TLUSTY_grid/wangluqian/TlustyB/BG*.flux')
dirout = '/share/lijiao/speedyfit/modelgrids/'
primary_hdu = get_primaryhdu(LOGZ='G', REF='BTLUSTY2006', Z = 0.)
table_hdus = []
table_hdus = joblib.Parallel(n_jobs=10, )(joblib.delayed(get_tablehdu)(fname, R =300, wave_start= 9.09e+01, wave_end = 1.60e+06) for fname in tqdm(filelist))
#for i, fname in enumerate(filelist):
#    table_hdu = get_tablehdu(fname)
#    table_hdus.append(table_hdu)
    
hdul = fits.HDUList([primary_hdu] + table_hdus)
hdul.writeto(dirout+'TlustyB_G_v2.fits', overwrite=True)

100%|██████████| 163/163 [00:24<00:00,  6.55it/s]


# check re.compiler

In [18]:
fname = '/share/lijiao/TLUSTY_grid/wangluqian/TlustyB/BG15000g475v2.flux'

line_ttt = '5.50794208D+16 2.5798-101 2.5798+101\n'
line_ttt = '5.50794208D+16 2.5798+101\n'
l0= re_dbl_fort.sub(r'\1e\2', line_ttt)
l = re_dbl_fort1.sub(r'\1e-\2', l0)
ll = re_dbl_fort2.sub(r'\1e+\2', l)
print(line_ttt)
ll

5.50794208D+16 2.5798+101



'5.50794208e+16 2.5798e+101\n'